In [1]:
pip install datasets scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [3]:
# Try loading a specific split of the dataset
dataset = load_dataset("google-research-datasets/go_emotions")

# Check the loaded data
print(dataset)

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})


In [10]:
# Memeriksa fitur dari dataset
print(dataset['train'].features)

# Menampilkan semua label yang ada dalam dataset
labels = dataset['train'].features['labels']
print("Label IDs:", labels)

# Jika label berupa ID, maka tampilkan nama label dari IDs tersebut
if isinstance(labels, dict) and 'feature' in labels:
    label_names = labels['feature'].names
    print("Label names:", label_names)
else:
    print("Labels tidak memiliki atribut 'names'")

{'text': Value(dtype='string', id=None), 'labels': Sequence(feature=ClassLabel(names=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'], id=None), length=-1, id=None), 'id': Value(dtype='string', id=None)}
Label IDs: Sequence(feature=ClassLabel(names=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'], id=None), length=-1, id=None)
Labels tidak memiliki atribut 'names'


In [ ]:
# Load the dataset from Hugging Face
dataset = load_dataset("google-research-datasets/go_emotions")

# Combine the training and validation data
data = dataset['train']
texts = data['text']
labels = data['labels']

# TF-IDF vectorizer
max_features = 10000  # Number of unique words to use
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
x_data = tfidf_vectorizer.fit_transform(texts).toarray()

# Convert labels to binary matrix
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_data = mlb.fit_transform(labels)

# K-Fold Cross Validation
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(x_data):
    x_train, x_val = x_data[train_index], x_data[val_index]
    y_train, y_val = y_data[train_index], y_data[val_index]
    
    # Create a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)
    print(f'Fold accuracy: {accuracy:.4f}')

# Calculate the mean accuracy
print(f'Mean accuracy: {np.mean(accuracies):.4f}')

Fold accuracy: 0.3075
Fold accuracy: 0.3006
